In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('/home/yurui/GDSC_2/GDSC2_curve_data/GDSC2_curve_estimate_4_para.csv')
dose_response_binary = pd.read_excel('/home/yurui/GDSC_2/GDSC2_dataset/TableS5C.xlsx', )

/home/yurui/anaconda3/envs/drug_response_env/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
dose_response_binary = dose_response_binary.iloc[4:, 1:]
## Set the first row as column names. Then drop the first row.
dose_response_binary.columns = dose_response_binary.iloc[0]
dose_response_binary = dose_response_binary.drop(dose_response_binary.index[0]).reset_index(drop=True)
drug_list = dose_response_binary.columns[1:]
drug_threshold = {name: dose_response_binary.iloc[0][name] for name in drug_list}
raw_dataset = pd.read_excel('/home/yurui/GDSC_2/GDSC2_dataset/GDSC2_fitted_dose_response_27Oct23.xlsx')

In [5]:
drug_columns = ['DRUG_ID', 'DRUG_NAME', 'PATHWAY_NAME']
cell_columns = ['CELL_LINE_NAME', 'TCGA_DESC','COSMIC_ID']
cell_meta = raw_dataset[cell_columns].drop_duplicates().reset_index(drop=True)
drug_meta = raw_dataset[drug_columns].drop_duplicates().reset_index(drop=True)
print(f'There are {cell_meta.COSMIC_ID.unique().__len__()} cell lines of {cell_meta.TCGA_DESC.unique().__len__()} cancer types.')
print(f'There are {drug_meta.DRUG_ID.unique().__len__()} drugs targeting {drug_meta.PATHWAY_NAME.unique().__len__()} pathways.')

There are 969 cell lines of 33 cancer types.
There are 295 drugs targeting 24 pathways.


In [ ]:
cell_meta['CANCER_TYPE'] = cell_meta['TCGA_DESC'].astype('category').cat.codes
drug_meta['PATHWAY_TYPE'] = drug_meta['PATHWAY_NAME'].astype('category').cat.codes
GE = pd.read_csv('../GDSC2_dataset/Cell_line_RMA_proc_basalExp.txt', sep='\t')
## Filter the rows with any NA values.
GE.dropna(axis=0, inplace=True)
GE_meta = GE[['GENE_SYMBOLS', 'GENE_title']].drop_duplicates().reset_index(drop=True)
GE_meta['Gene_code'] = GE_meta.index
GE_meta.to_csv('../GDSC2_dataset/meta_data/GE_meta.csv', index=False)
GE.drop(['GENE_title'], axis=1, inplace=True)
GE.rename(columns={col: col.split('.')[1] if len(col.split('.')) > 1 else col for col in GE.columns[1:]}, inplace=True)
GE.set_index('GENE_SYMBOLS', inplace=True)
GE_final = GE.T
GE_final.index.name = 'COSMIC_ID'
GE_final.to_csv('../GDSC2_dataset/CL_feature/GE_final.csv')

In [ ]:
drug_data = pd.read_csv('..//GDSC2_dataset/Drug_feature/PubChem_compound.csv')
drug_data.rename(columns={'cid': 'DRUG_ID'}, inplace=True)
drug_data.rename(columns={'canonicalsmiles': 'SMILES'}, inplace=True)
drug_meta = pd.merge(drug_data[['DRUG_ID','SMILES']], drug_meta, on='DRUG_ID')
drug_meta.to_csv('../GDSC2_dataset/meta_data/drug_meta.csv', index=False)